In [1]:
import requests as rq
import pandas as pd
from bs4 import BeautifulSoup
import re
import time
from tqdm import tqdm, tqdm_notebook, notebook

In [2]:
url = "https://www.diagnos.ru/procedures/manipulation"

In [3]:
headers = {"Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
           "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.63 Safari/537.36"
          }

In [4]:
response = rq.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')
soup;

In [5]:
# Парсим ссылки на процедуры
pattern = 'padding-left:[0-9]*px; padding-top:8px'
procedures = soup.find_all('div', {'style': re.compile(pattern)})
links = []
for procedure in range(len(procedures)):
    link = procedures[procedure].find('a').get('href')
    name = procedures[procedure].find('a').text.strip()
    links.append(link)
links;

In [6]:
# Задаем ключевые слова, по которым будем метчить необходимую информацию
required = ("Подготовка","Показания","Осложнения","Побочные")

In [7]:
info = {}
with notebook.tqdm(desc="Link", total=len(links)) as pbar_outer:
    
    counter = 0 # Счетчик успешных итераций парсера
    
    for number, link in enumerate(links): # Итерируемся по нашим собранным ранее ссылкам
        
        try:
            # Получаем информацию со страницы
            url = f'https://www.diagnos.ru{link}'
            response = rq.get(url, headers=headers)
            soup = BeautifulSoup(response.text, 'html.parser')
            
            dct = {}
            description = '-'
            
            name = soup.find('h1').text.strip() #Получаем название процедуры
            dct['Название процедуры'] = name
            
            overall = soup.find('div',{'id':'aktar_content'}) #Ищем всю информацию из статьи
            
            # Убираем мешающий дальнейшему парсингу тег
            if overall.find('p', {"class": "author"}) is not None: 
                overall.find('p', {"class": "author"}).decompose()
            
            # Получаем общее описание процедуры
            first = re.search('<div id="aktar_content">', str(overall)).span()[0] # От какого элемента делаем срез
            second = re.search('h2', str(overall)).span()[1] # До какого элемента делаем срез
            #second+1: в питоне правая граница не включается, в данном случае это какой-то символ из текста, для того, чтобы добавить его для удобства обработки мы и пишем +1, чтобы наш интервал включал правую границу
            desc = str(overall)[first:second+1]
            # Приводим к нужному виду
            #xa0: это неразрывный пробел в html, назначение — запрет переноса строки по словам в месте, где он стоит. Мы убираем его, потому что он портит вид нашего текста
            description = BeautifulSoup(desc, 'html.parser').text.strip('<>\n').replace('\n',' ').replace('\xa0',' ')
            dct['Описание'] = description
            
            #Получаем описание заголовков: "Подготовка", "Показания", "Осложнения" -- если они существуют
            to_parse = str(overall).split('h2')[1:] #Разделяем нашу статью по заголовкам
            for element in range(1,len(to_parse),2):
                for requ in required:
                    # Если в заголовке есть ключевое слово, парсим его
                    if requ in to_parse[element-1]:
                        souped =  BeautifulSoup(to_parse[element]).text.strip('></').replace('\n', ' ').replace('\xa0',' ').strip()
                        if 'источники' in souped.lower(): # Убираем ненужный элемент -- источники
                            souped = souped[:souped.lower().find('источники')].strip()
                        if requ == "Побочные":
                            dct["Осложнения"] = souped
                            continue
                        else:
                            dct[requ] = souped
                    if requ not in dct and requ != "Побочные":
                        dct[requ] = '-'
            
            info[number] = dct # Записываем необходимую информацию о процедуре в словарь
            
            status = 'Success' # Если все в порядке, то информацию о процедуре была успешно получена
            
        except Exception as Ex:
            status = Ex # Если все не в порядке, то информацию о процедуре не была получена
            pass
        
        if status == 'Success': # Записываем количество успешно полученных процедур
            counter += 1
            
        pbar_outer.update(1)
        print(f'{number+1}/{len(links)}\nURL: {link}\nStatus: {status}\n') # Добавляем вывод, для возможной отладки кода
        print('--------------------------------------------')
        
        # Иногда стоит делать задержку между итерациями, чтобы не словить бан за спам, но на этом сайте такой зашиты нет(вроде как)
        # time.sleep(0.5)
        # tqdm — библиотека, позволяющая добавлять в код шкалу прогресса, что бывает удобно в таких парсерах (нежели не знать сколько вообще осталось до полного парсинга)
    tqdm.write(f'{counter}/{len(links)} URLs successfully parsed')
        

Link:   0%|          | 0/63 [00:00<?, ?it/s]

1/63
URL: /procedures/manipulation/gisteroscopia
Status: Success

--------------------------------------------
2/63
URL: /procedures/manipulation/irrigoscopia
Status: Success

--------------------------------------------
3/63
URL: /procedures/manipulation/kolonoskopia
Status: Success

--------------------------------------------
4/63
URL: /procedures/manipulation/laparoskopia-gin
Status: Success

--------------------------------------------
5/63
URL: /procedures/manipulation/gsg
Status: Success

--------------------------------------------
6/63
URL: /procedures/manipulation/kolposkopia
Status: Success

--------------------------------------------
7/63
URL: /procedures/manipulation/gastroskopia
Status: Success

--------------------------------------------
8/63
URL: /procedures/manipulation/duktografia
Status: Success

--------------------------------------------
9/63
URL: /procedures/manipulation/amniocentes
Status: Success

--------------------------------------------
10/63
URL: /proce

In [8]:
info # Полученные данные в виде словаря

{0: {'Название процедуры': 'Гистероскопия',
  'Описание': 'Гистероскопией называется метод эндоскопического исследования в гинекологии, позволяющий при помощи особого инструмента проникнуть в полость матки через ее шейку и при помощи вмонтированной на конце гистероскопа камеры визуально оценить состояние полости матки и области маточных труб. Данный метод широко применяется не только для диагностики, но и для одновременного лечения некоторых заболеваний матки, а также позволяет брать образцы ткани для гистологического исследования.',
  'Подготовка': '-',
  'Показания': 'Гистроскопия позволяет провести диагностику:  бесплодия, невынашивания беременности, кровотечений, опухолевых процессов, неоплазий (патологического изменения тканей вплоть до онкологии), полипов и миом, спаек или перегородок внутри матки, аномалий строения матки, произвести прижигание сосудов или удаление некоторых образований.',
  'Осложнения': 'Гистероскопия – это инвазивная процедура с проникновением аппарата внутрь 

In [9]:
pd.DataFrame(info).T.to_excel('parsed.xlsx', index=False) # Полученные данных в виде таблицы Excel